In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.covariance import graphical_lasso, empirical_covariance
from graph import draw_graph, Node, get_graph
from data import get_nodes, get_scaled_data

In [2]:
nodes, nodes_ids = get_nodes()
df = get_scaled_data()

In [3]:
numerical_columns = df.columns[1:]
X = df[numerical_columns]
S = empirical_covariance(X)

In [55]:
m,n = np.shape(X)
L = cp.Variable((n,n), "L")
Y = X.to_numpy().transpose()
#Y = cp.Variable((n,n), 'Y')

Alpha = 0.0025
Beta = 0.5

In [59]:
# ineq is matrix of zeroes
# cp.norm(X-Y, "fro")**2+ deleted because X=Y so 0
# (np.ones((n,n))- np.identity(n))
# First fix Y as X
# With learned L, fix L and optimize for variable Y
# cp.quad_form(Y, L)
obj = cp.Minimize(Alpha*cp.trace(cp.quad_form(Y,L))+Beta*cp.norm(L,"fro")**2)
const = [ cp.trace(L) == n, 
                   L==L.T,
                   #L@(np.ones((n,n))- np.identity(n))<=0,
                     L@np.ones((n,1)) == 0,
                  ]

for i in range(n):
    for j in range(n):
        if i != j:
            const.append(L[i,j] <= 0)

prob = cp.Problem(obj, const)
prob.solve(solver=cp.ECOS)

12.326768530526996

In [60]:
print(prob)

minimize 0.0025 @ trace([[-0.81 -1.40 ... -0.27 -0.18]
 [-1.16 -1.56 ... 0.17 0.17]
 ...
 [-1.33 -0.43 ... -0.11 -0.51]
 [-0.31 -1.05 ... 0.54 0.70]].T @ L @ [[-0.81 -1.40 ... -0.27 -0.18]
 [-1.16 -1.56 ... 0.17 0.17]
 ...
 [-1.33 -0.43 ... -0.11 -0.51]
 [-0.31 -1.05 ... 0.54 0.70]]) + 0.5 @ power(Pnorm(reshape(L, (64,), F), 2), 2.0)
subject to trace(L) == 8.0
           L == L.T
           L @ [[1.00]
 [1.00]
 ...
 [1.00]
 [1.00]] == 0.0
           L[0, 1] <= 0.0
           L[0, 2] <= 0.0
           L[0, 3] <= 0.0
           L[0, 4] <= 0.0
           L[0, 5] <= 0.0
           L[0, 6] <= 0.0
           L[0, 7] <= 0.0
           L[1, 0] <= 0.0
           L[1, 2] <= 0.0
           L[1, 3] <= 0.0
           L[1, 4] <= 0.0
           L[1, 5] <= 0.0
           L[1, 6] <= 0.0
           L[1, 7] <= 0.0
           L[2, 0] <= 0.0
           L[2, 1] <= 0.0
           L[2, 3] <= 0.0
           L[2, 4] <= 0.0
           L[2, 5] <= 0.0
           L[2, 6] <= 0.0
           L[2, 7] <= 0.0
           

In [61]:
print("\nThe optimal value is", prob.value)
print("A solution Theta is")
print(L.value)
print(np.sum(np.trace(L.value)))
res = L.value
for i in range(n):
    for j in range(n):
        if -res[i][j] < 0.0001:
            res[i][j] = 0
print(res)


The optimal value is 12.326768530526996
A solution Theta is
[[ 1.12997502e+00 -1.91196608e-01 -8.05417006e-01 -1.18581317e-09
  -1.64731087e-09 -6.90312603e-02 -6.43301417e-02 -2.07073811e-09]
 [-1.91196608e-01  9.56790632e-01 -6.10204746e-02 -5.30249784e-02
  -9.04485661e-02 -7.36891307e-02 -4.87410841e-01 -3.36229044e-08]
 [-8.05417006e-01 -6.10204746e-02  8.93351026e-01 -9.80924881e-10
  -1.05014600e-09 -2.69135385e-02 -3.67701872e-09 -1.43498316e-09]
 [-1.18565771e-09 -5.30249784e-02 -9.80718242e-10  9.69297885e-01
  -4.77197573e-01 -6.10884906e-09 -1.68142929e-01 -2.70932397e-01]
 [-1.64722918e-09 -9.04485661e-02 -1.05001341e-09 -4.77197573e-01
   1.31011188e+00 -3.94631936e-02 -2.75155102e-01 -4.27847445e-01]
 [-6.90312603e-02 -7.36891307e-02 -2.69135385e-02 -6.10880290e-09
  -3.94631936e-02  4.79813123e-01 -3.31719203e-09 -2.70715990e-01]
 [-6.43301417e-02 -4.87410841e-01 -3.67683619e-09 -1.68142929e-01
  -2.75155102e-01 -3.31727682e-09  1.14310179e+00 -1.48062772e-01]
 [-2.070